In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

## Step A - Multiple Product Detection

In [ ]:
query_images = ['models/0.jpg','models/1.jpg','models/11.jpg','models/19.jpg','models/24.jpg','models/25.jpg','models/26.jpg']
scene_images = ['scenes/e1.png','scenes/e2.png','scenes/e3.png','scenes/e4.png','scenes/e5.png']

In [ ]:
MIN_MATCH_COUNT = 100

for scene in scene_images:
    
    img2_bgr = cv2.imread(scene)
    img2 = cv2.cvtColor(img2_bgr,cv2.COLOR_BGR2RGB)
    
    print("\n Scene: " + scene)
    
    plt.imshow(img2),plt.show();
    
    for img in query_images:
        img1_bgr = cv2.imread(img)
        img1 = cv2.cvtColor(img1_bgr,cv2.COLOR_BGR2RGB)
        img2_bgr = cv2.imread(scene)
        img2 = cv2.cvtColor(img2_bgr,cv2.COLOR_BGR2RGB)

        sift = cv2.xfeatures2d.SIFT_create()

        kp1, des1 = sift.detectAndCompute(img1,None)
        kp2, des2 = sift.detectAndCompute(img2,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des1,des2,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)

        if len(good)>MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
            h,w,d = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)
            img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
            draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matchesMask, # draw only inliers
                           flags = 2)
            print( "Product " + img + ": enough matches are found!!! - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
        #    plt.figure(figsize = (15,15))
            plt.imshow(img3),plt.show();
        else:
            print( "Product " + img + ": not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            matchesMask = None